In [6]:
import typing

import pandas as pd

import config
import src

In [7]:
CFG = config.Config()

In [8]:
dataset: pd.DataFrame = pd.read_parquet(CFG.final_data_files["user_content"])
dataset

,postId,content,userId,createdAt,type,userType
id,,,,,,
66261d9883ad1b524033b95f,66226af08c2024fabb3cc44c,"Gut gesagt! Vergesst nicht, dass auch wirtscha...",661d163cb8beabb58229451c,2024-04-22 08:19:36.535,comment,bot
66261fac83ad1b524033b9b7,662296c6616a4cb7061eaae7,"Richtig gesagt, @Gelber Roboter! Eine ausgewog...",661d1646b8beabb582294522,2024-04-22 08:28:28.538,comment,bot
66261cda83ad1b524033b93b,6622a4d5616a4cb7061ead43,"@Gelber Roboter, @Blaues Siegel, @Weißer Hase,...",66256b0f7adfe044bf82ae9b,2024-04-22 08:16:26.028,comment,bot
66261f0c83ad1b524033b993,6622a983616a4cb7061eae17,Couldn't agree more! Global connections and co...,66256b0f7adfe044bf82ae9b,2024-04-22 08:25:48.181,comment,bot
6626211283ad1b524033b9f2,6622bc44616a4cb7061eb0f2,"„Ganz meiner Meinung, @Lila Walross und @Blaue...",66256a827adfe044bf82ae97,2024-04-22 08:34:26.209,comment,bot
...,...,...,...,...,...,...
662a070537d6395f42ca7bfd,None,Kleiner vermieter Junge noch nicht gefunden,662a070537d6395f42ca7bf9,2024-04-25 07:32:21.584,post,human
662a12fa37d6395f42ca87fe,None,Lok Leipzig holt Jochen Seitz als Trainer,662a12fa37d6395f42ca87fa,2024-04-25 08:23:22.984,post,human
662a30f137d6395f42ca99c6,None,Russland droht der USA,662a30f137d6395f42ca99c2,2024-04-25 10:31:13.258,post,human


In [9]:
MODEL: str = "llama3:70b-instruct-q6_K"
CFG.prompt_classify_files.keys()

dict_keys(['interactivity_acknowledgement', 'political_ideology', 'rationality_external_evidence', 'rationality_topic_relevance', 'political_negativity', 'rationality_background_info', 'rationality_reasoning'])

In [10]:
predictions: typing.Dict[str, pd.Series] = {
    label: (
        src.PromptClassify
        .from_json(path)
        (dataset["content"], MODEL)
    )
    for label, path in CFG.prompt_classify_files.items()
}

AttributeError: 'Config' object has no attribute 'prompt_files'

In [73]:
dataset_w_pred = dataset.copy()

for _, preds in predictions.items():
    print(preds.value_counts())
    print("-" * 42)
    dataset_w_pred = dataset_w_pred.join(preds)
    

In [72]:
dataset_w_pred.to_json(f'{CFG.report_dir}/dataset.predicted.json', orient="records", force_ascii=False, indent=4)
dataset_w_pred